In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  
from IPython.display import HTML

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, precision_score, recall_score
import pandas
import numpy as np
import json
import matplotlib.pyplot as plt
import os

pandas.options.display.float_format = '{:,.3f}'.format

In [ ]:
from evaluate import load, get_results, get_labels, transform_binary_probabilities, transform_binary_predictions, calculate_accuracy_loss, plot_confusion_matrix, plot_precision_recall, plot_roc_curve, calculate_pr_auc, calculate_confusion_matrix_stats, calculate_confusion_matrix, plot_tsne, accession_from_filename, plot_expert_confusion, plot_multiple_grad_cam, get_expert_results

In [ ]:
from data import describe, all_features, data, print_describe, load_single
from run import characterize_data
from config import config

# Parameters

In [ ]:
DIRECTORY = "c3-c4-free/data"
MODEL_UUID = "1b356a9e-126c-4f91-8f41-8d6baa4f79de"
MODEL_TYPE = "v2"
SELECTIONS = "None"
PARAMETERS_ID = "parameters"

In [ ]:
SELECTIONS = eval(SELECTIONS)
FEATURES = "{}/features.csv".format(DIRECTORY)
RAW = "{}/raw".format(DIRECTORY)
TRAIN = "{}/train".format(DIRECTORY)
VALIDATION = "{}/validation".format(DIRECTORY)
MODEL = "{}/models/{}-{}.h5".format(config.OUTPUT, MODEL_UUID, MODEL_TYPE)

## Lesion descriptions

In [ ]:
print_describe(raw=RAW, features=FEATURES)

In [ ]:
train, validation, _ = data(shuffle_train=False, validation_dir=VALIDATION, train_dir=TRAIN)

In [ ]:
print("training data breakdown: ", characterize_data(train))
print("validation data breakdown: ", characterize_data(validation))

In [ ]:
model = load(MODEL)

In [ ]:
results = get_results(model, validation)

In [ ]:
probabilities = transform_binary_probabilities(results)
predictions = transform_binary_predictions(results)
labels = get_labels(validation)

## Accuracy and loss

In [ ]:
loss, accuracy = calculate_accuracy_loss(model, validation)
print("Accuracy:", accuracy)
print("Loss:", loss)

## Confusion matrix

In [ ]:
fig = plot_confusion_matrix(validation, results)
fig.savefig("figures/{}-confusion-matrix.svg".format(PARAMETERS_ID), bbox_inches = "tight")

In [ ]:
stats = calculate_confusion_matrix_stats(labels, results)
pandas.DataFrame(stats, index=validation.class_indices)

## ROC Curve standalone

In [ ]:
plot_roc_curve(labels, results)

In [ ]:
print("ROC AUC:", roc_auc_score(labels, probabilities))

## Precision Recall

In [ ]:
print("Average precision:", average_precision_score(labels, probabilities))
print("Precision:", precision_score(labels, predictions), "Recall:", recall_score(labels, predictions))

In [ ]:
fig = plot_precision_recall(labels, results)
fig.savefig("figures/{}-precision-recall.svg".format(PARAMETERS_ID), bbox_inches = "tight")

In [ ]:
print("PR AUC:", calculate_pr_auc(labels, results))

## F1 Score

In [ ]:
print("F1 score:", f1_score(labels, predictions))

## TSNE

In [ ]:
malignancy, modality, category = all_features(features=FEATURES)

In [ ]:
tsne_dataset = train

In [ ]:
tsne_labels = [malignancy.get(accession_from_filename(f), "unknown") for f in tsne_dataset.filenames]

In [ ]:
fig = plot_tsne(model, "dense_5", tsne_dataset, tsne_labels, perplexity=5)
fig.savefig("figures/{}-tsne.svg".format(PARAMETERS_ID), bbox_inches = "tight")

## Experts

In [ ]:
expert_1, fig = plot_expert_confusion("evaluations/radiology-9809.json", validation)
fig.savefig("figures/{}-confusion-matrix-expert-1.svg".format(PARAMETERS_ID), bbox_inches = "tight")

In [ ]:
pandas.DataFrame(expert_1, index=validation.class_indices)

## ROC with experts

In [ ]:
fig = plot_roc_curve(labels, results, experts=[{ **expert_1, "name":"expert 1" }, { **expert_1, "name":"expert 2" }])
fig.savefig("figures/{}-roc-curve.svg".format(PARAMETERS_ID), bbox_inches = "tight")

## Grad Cam

In [ ]:
inv_indices = {v: k for k, v in validation.class_indices.items()}
for k, v in SELECTIONS.items(): 
    display(HTML("<h5>diagnosis: {} | prediction: {}</h5>".format(inv_indices[k[0]], inv_indices[k[1]])))
    fig = plot_multiple_grad_cam(
        [os.path.join(VALIDATION, image) for image in v], 
        model, 
        "dense_6", 
        penultimate_layer="res5c_branch2c",
        filter_idx=[0],
    )
    fig.savefig("figures/{}-grad-cam-{}-{}.png".format(PARAMETERS_ID, inv_indices[k[0]], inv_indices[k[1]]), bbox_inches = "tight")
    print(get_expert_results("evaluations/radiology-9809.json", v, expert_key="malignantBenign"))

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')